# Comprehensive Parameter Sweep Results

## Experiment Overview

This notebook performs a comprehensive parameter sweep with:

- **5 Scenarios** - Different obstacle configurations and initial conditions
- **3 Planners** - MPPI-Particle, MPPI-Worst, Probabilistic
- **3 Gamma values** - [1, 3, 6, 10]
- **3 Epsilon values** - [0.01, 0.05, 0.1, 0.2, 0.5] 

**Total: xx experiments**

## Key Metrics Tracked

1. **min_dist** - Minimum distance to obstacles during execution (higher is safer)
2. **time_to_goal** - Time taken to reach the goal (lower is faster)

## Outputs Generated

1. **parameter_sweep_results.csv** - Complete results table
2. **min_dist_comparison.png** - Bar plots comparing min_dist across planners
3. **time_to_goal_comparison.png** - Bar plots comparing time_to_goal across planners
4. **min_dist_heatmap.png** - Heatmaps showing gamma vs epsilon effects on min_dist
5. **time_to_goal_heatmap.png** - Heatmaps showing gamma vs epsilon effects on time_to_goal

## Cell Guide

- **Cell 0**: Run all experiments (takes time!)
- **Cell 1**: Extract metrics and create summary DataFrame
- **Cell 2**: Visualize min_dist comparisons
- **Cell 3**: Visualize time_to_goal comparisons  
- **Cell 4**: Create heatmaps showing parameter effects
- **Cell 5**: Show detailed comparison tables
- **Cell 6**: Access and inspect specific experiment results

## How to Use

1. Run Cell 0 to execute all experiments
2. Run Cells 1-5 to generate all analysis and visualizations
3. Modify Cell 6 to inspect specific experiments in detail


In [ ]:
from experiments import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === COMPREHENSIVE PARAMETER SWEEP ===
# Test over: 5 scenarios × 3 planners × a gamma × b epsilon = 15 × a × b experiments

gamma_values = [1, 3, 6, 10]
epsilon_values = [0.01, 0.05, 0.1, 0.2, 0.5]  # alpha parameter in config
gamma_values = [1]
epsilon_values = [0.01]  # alpha parameter in config

# Define five scenario configurations
scenarios = [
    {
        "name": "Right-Straight",
        "obstacle_goals": [[-2.0, 0.0], [0.0, -2.0], [0.0, 2.0]],
        "obs_init": [2.0, 0.0],
        "target_goal": [-2.0, 0.0]
    },
    {
        "name": "Right-Straight (unknown)",
        "obstacle_goals": [[0.0, -2.0], [0.0, 2.0]],
        "obs_init": [2.0, 0.0],
        "target_goal": [-2.0, 0.0]
    },
    {
        "name": "Down-Left",
        "obstacle_goals": [[-1.0, 2.0], [1.0, 2.0]],
        "obs_init": [0.0, -1.0],
        "target_goal": [-1.0, 2.0]  # obstacle_goals[0]
    },
    {
        "name": "Down-Right",
        "obstacle_goals": [[-1.0, 2.0], [1.0, 2.0]],
        "obs_init": [0.0, -1.0],
        "target_goal": [1.0, 2.0]  # obstacle_goals[1]
    },
    {
        "name": "Down-Straight (unknown)",
        "obstacle_goals": [[-1.0, 2.0], [1.0, 2.0]],
        "obs_init": [0.0, -1.0],
        "target_goal": [0.0, 2.0]
    }
]

# Define three planner configurations
planner_configs = [
    {"name": "Particle", "planner_type": "MPPI", "dynamics_type": "Predict_TV"},
    {"name": "Worst", "planner_type": "MPPI", "dynamics_type": "Worst"},
    {"name": "CC-MPC", "planner_type": "Probabilistic", "dynamics_type": None}
]

results_sweep = {}

# Calculate actual experiments needed (optimized)
# Particle: all gamma × epsilon combinations
# Worst: once per gamma (epsilon doesn't matter)
# CC-MPC: once per epsilon (gamma doesn't matter)
actual_experiments = len(scenarios) * (
    len(gamma_values) * len(epsilon_values) +  # Particle
    len(gamma_values) +                         # Worst (once per gamma)
    len(epsilon_values)                         # CC-MPC (once per epsilon)
)
total_results = len(scenarios) * len(planner_configs) * len(gamma_values) * len(epsilon_values)

print(f"COMPREHENSIVE PARAMETER SWEEP (OPTIMIZED)")
print(f"=" * 100)
print(f"Scenarios: {len(scenarios)}")
print(f"Planners: {[cfg['name'] for cfg in planner_configs]}")
print(f"Gamma values: {gamma_values}")
print(f"Epsilon (alpha) values: {epsilon_values}")
print(f"Actual experiments to run: {actual_experiments} (optimized)")
print(f"Total results populated: {total_results}")
print(f"Metrics tracked: min_dist, time_to_goal")
print(f"")
print(f"Optimization notes:")
print(f"  - Worst planner: varies with gamma, independent of epsilon → run once per gamma")
print(f"  - CC-MPC planner: varies with epsilon, independent of gamma → run once per epsilon")
print(f"  - Particle planner: depends on both gamma and epsilon → run all combinations")
print(f"=" * 100)

experiment_count = 0

for scenario in scenarios:
    scenario_name = scenario["name"]
    
    print(f"\n{'#'*100}")
    print(f"# SCENARIO: {scenario_name}")
    print(f"#   obstacle_goals: {scenario['obstacle_goals']}")
    print(f"#   obs_init: {scenario['obs_init']}")
    print(f"#   target_goal: {scenario['target_goal']}")
    print(f"{'#'*100}")
    
    for planner_cfg in planner_configs:
        planner_name = planner_cfg["name"]
        planner_type = planner_cfg["planner_type"]
        dynamics_type = planner_cfg["dynamics_type"]
        
        print(f"\n{'='*100}")
        print(f"PLANNER: {planner_name} (type={planner_type}, dynamics={dynamics_type})")
        print(f"{'='*100}")
        
        if planner_name == "Worst":
            # Worst planner: varies with gamma, independent of epsilon
            print(f"  NOTE: Worst planner varies with gamma but is independent of epsilon")
            for gamma in gamma_values:
                experiment_count += 1
                epsilon_run = epsilon_values[0]  # Use first epsilon (doesn't matter)
                
                print(f"\n[{experiment_count}/{actual_experiments}] {scenario_name} | {planner_name} | gamma={gamma} | eps=ALL")
                print("-" * 100)
                
                config = get_config_custom(
                    planner_type="MPPI",
                    dynamics_type=dynamics_type,
                    gamma=gamma,
                    alpha=epsilon_run,  # epsilon doesn't matter for Worst
                    obstacle_goals=scenario["obstacle_goals"],
                    obs_init=scenario["obs_init"],
                    target_goal=scenario["target_goal"],
                    T_max=50,
                    dt_plan=0.1,
                    dt_exec=0.1,
                    cvar_weight=1000.0,
                    particle_count=1,
                    save_plots=False
                )
                
                # Run experiment once for this gamma
                result = run_experiment(config)
                
                # Store result for ALL epsilon values with this gamma
                for eps in epsilon_values:
                    key = f"{scenario_name}_{planner_name}_gamma{gamma}_eps{eps}"
                    results_sweep[key] = result
                
                # Display metrics
                if isinstance(result, dict):
                    min_dist = result.get('min_distance', 'N/A')
                    time_to_goal = result.get('time_to_goal', 'N/A')
                    print(f"✅ Completed | min_dist: {min_dist} | time_to_goal: {time_to_goal}")
                    print(f"   Copied to all ε values: {epsilon_values}")
                
        elif planner_name == "CC-MPC":
            # CC-MPC planner: varies with epsilon, independent of gamma
            print(f"  NOTE: CC-MPC planner varies with epsilon but is independent of gamma")
            for epsilon in epsilon_values:
                experiment_count += 1
                gamma_run = gamma_values[0]  # Use first gamma (doesn't matter)
                
                print(f"\n[{experiment_count}/{actual_experiments}] {scenario_name} | {planner_name} | gamma=ALL | eps={epsilon}")
                print("-" * 100)
                
                config = get_config_custom(
                    planner_type="Probabilistic",
                    dynamics_type=None,
                    gamma=gamma_run,  # gamma doesn't matter for CC-MPC
                    obstacle_goals=scenario["obstacle_goals"],
                    obs_init=scenario["obs_init"],
                    target_goal=scenario["target_goal"],
                    T_max=50,
                    collision_threshold=epsilon,
                    u_sigma=4.0,
                    dt_plan=0.1,
                    dt_exec=0.1,
                    cvar_weight=1000.0,
                    particle_count=1,
                    save_plots=False
                )
                
                # Run experiment once for this epsilon
                result = run_experiment(config)
                
                # Store result for ALL gamma values with this epsilon
                for gamma in gamma_values:
                    key = f"{scenario_name}_{planner_name}_gamma{gamma}_eps{epsilon}"
                    results_sweep[key] = result
                
                # Display metrics
                if isinstance(result, dict):
                    min_dist = result.get('min_distance', 'N/A')
                    time_to_goal = result.get('time_to_goal', 'N/A')
                    print(f"✅ Completed | min_dist: {min_dist} | time_to_goal: {time_to_goal}")
                    print(f"   Copied to all γ values: {gamma_values}")
                
        else:
            # Particle planner: depends on both gamma and epsilon
            for gamma in gamma_values:
                for epsilon in epsilon_values:
                    experiment_count += 1
                    print(f"\n[{experiment_count}/{actual_experiments}] {scenario_name} | {planner_name} | gamma={gamma} | eps={epsilon}")
                    print("-" * 100)
                    
                    config = get_config_custom(
                        planner_type="MPPI",
                        dynamics_type=dynamics_type,
                        gamma=gamma,
                        alpha=epsilon,
                        obstacle_goals=scenario["obstacle_goals"],
                        obs_init=scenario["obs_init"],
                        target_goal=scenario["target_goal"],
                        T_max=50,
                        cvar_weight=1000.0,
                        particle_count=256,
                        dt_plan=0.1,
                        dt_exec=0.1,
                        save_plots=False
                    )
                    
                    # Run experiment
                    result = run_experiment(config)
                    
                    # Store results
                    key = f"{scenario_name}_{planner_name}_gamma{gamma}_eps{epsilon}"
                    results_sweep[key] = result
                    
                    # Display metrics
                    if isinstance(result, dict):
                        min_dist = result.get('min_distance', 'N/A')
                        time_to_goal = result.get('time_to_goal', 'N/A')
                        print(f"✅ Completed | min_dist: {min_dist} | time_to_goal: {time_to_goal}")
                    else:
                        print(f"✅ Completed: {key}")

print("\n" + "=" * 100)
print("PARAMETER SWEEP COMPLETED!")
print(f"Total experiments run: {len(results_sweep)}")
print("\nSummary by scenario:")
for scenario in scenarios:
    scenario_name = scenario["name"]
    count = sum(1 for key in results_sweep.keys() if key.startswith(scenario_name))
    print(f"  {scenario_name}: {count} experiments")


In [ ]:
# === EXTRACT AND ANALYZE min_dist AND time_to_goal ===

# Create comprehensive summary dataframe
summary_data = []

for key, result in results_sweep.items():
    # Parse key: "ScenarioN_PlannerName_gammaN_epsN.N"
    parts = key.split('_')
    
    # Extract scenario
    scenario = parts[0]
    
    # Extract planner (handle hyphenated names)
    if parts[1] == "MPPI-Predict":
        planner = "MPPI-Predict"
        gamma_part = parts[2]
        eps_part = parts[3]
    elif parts[1] == "MPPI-Worst":
        planner = "MPPI-Worst"
        gamma_part = parts[2]
        eps_part = parts[3]
    elif parts[1] == "Probabilistic":
        planner = "Probabilistic"
        gamma_part = parts[2]
        eps_part = parts[3]
    else:
        planner = parts[1]
        gamma_part = parts[2]
        eps_part = parts[3]
    
    gamma = float(gamma_part.replace('gamma', ''))
    epsilon = float(eps_part.replace('eps', ''))
    
    # Extract metrics
    if isinstance(result, dict):
        min_dist = result.get('min_distance', None)
        time_to_goal = result.get('time_to_goal', None)
    else:
        min_dist = None
        time_to_goal = None
    
    summary_data.append({
        'scenario': scenario,
        'planner': planner,
        'gamma': gamma,
        'epsilon': epsilon,
        'min_dist': min_dist,
        'time_to_goal': time_to_goal,
        'key': key
    })

df_results = pd.DataFrame(summary_data)
df_results = df_results.sort_values(['scenario', 'planner', 'gamma', 'epsilon'])

print("=" * 120)
print("COMPREHENSIVE RESULTS: min_dist and time_to_goal")
print("=" * 120)
print(df_results.to_string(index=False))

# Summary statistics
print("\n\n" + "=" * 120)
print("SUMMARY STATISTICS BY PLANNER")
print("=" * 120)

for planner in df_results['planner'].unique():
    planner_df = df_results[df_results['planner'] == planner]
    print(f"\n{planner}:")
    print(f"  min_dist   - Mean: {planner_df['min_dist'].mean():.4f}, Std: {planner_df['min_dist'].std():.4f}, Min: {planner_df['min_dist'].min():.4f}, Max: {planner_df['min_dist'].max():.4f}")
    print(f"  time_to_goal - Mean: {planner_df['time_to_goal'].mean():.4f}, Std: {planner_df['time_to_goal'].std():.4f}, Min: {planner_df['time_to_goal'].min():.4f}, Max: {planner_df['time_to_goal'].max():.4f}")

# Save to CSV for later analysis
csv_filename = "parameter_sweep_results.csv"
df_results.to_csv(csv_filename, index=False)
print(f"\n✅ Results saved to: {csv_filename}")


In [ ]:
# === VISUALIZE RESULTS: min_dist BY PLANNER AND SCENARIO ===

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

# Define scenario order
scenario_order = ['Right-Straight', 'Right-Straight (unknown)', 'Down-Left', 'Down-Right', 'Down-Straight (unknown)']
scenarios_to_plot = [s for s in scenario_order if s in df_results['scenario'].values]

# Create subplots for each scenario
n_scenarios = len(scenarios_to_plot)
fig, axes = plt.subplots(1, n_scenarios, figsize=(6*n_scenarios, 5))

if n_scenarios == 1:
    axes = [axes]

for idx, scenario in enumerate(scenarios_to_plot):
    ax = axes[idx]
    scenario_df = df_results[df_results['scenario'] == scenario]
    
    # Find min and max for THIS scenario only (for consistent y-axis within scenario)
    scenario_min = scenario_df['min_dist'].min()
    scenario_max = scenario_df['min_dist'].max()
    y_margin = (scenario_max - scenario_min) * 0.1
    scenario_ylim = [max(0, scenario_min - y_margin), scenario_max + y_margin]
    
    # Create grouped bar plot
    x = np.arange(len(gamma_values) * len(epsilon_values))
    width = 0.25
    
    # Define planner order: Particle, Worst, CC-MPC
    planner_order = ['Particle', 'Worst', 'CC-MPC']
    planners = [p for p in planner_order if p in scenario_df['planner'].values]
    
    for p_idx, planner in enumerate(planners):
        planner_df = scenario_df[scenario_df['planner'] == planner].sort_values(['gamma', 'epsilon'])
        positions = x + (p_idx - 1) * width
        
        ax.bar(positions, planner_df['min_dist'], width, label=planner, alpha=0.8)
    
    # Set labels and title
    ax.set_xlabel('Gamma & Epsilon', fontsize=10)
    ax.set_ylabel('min_distance', fontsize=10)
    ax.set_title(f'{scenario}\n[y: {scenario_ylim[0]:.2f} - {scenario_ylim[1]:.2f}]', 
                 fontsize=12, fontweight='bold')
    ax.legend()
    
    # Set y-axis range specific to this scenario
    ax.set_ylim(scenario_ylim)
    
    # Create x-tick labels
    labels = [f'γ={g}, ε={e}' for g in gamma_values for e in epsilon_values]
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax.grid(axis='y', alpha=0.3)

# Add overall title
fig.suptitle('Minimum Distance', fontsize=22, fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space at top for title
plt.savefig('min_dist_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ min_dist comparison plot saved as 'min_dist_comparison.png'")
print(f"   (Each scenario has its own y-axis range)")


In [ ]:
# === VISUALIZE RESULTS: time_to_goal BY PLANNER AND SCENARIO ===

# Define scenario order
scenario_order = ['Right-Straight', 'Right-Straight (unknown)', 'Down-Left', 'Down-Right', 'Down-Straight (unknown)']
scenarios_to_plot = [s for s in scenario_order if s in df_results['scenario'].values]

# Create subplots for each scenario
n_scenarios = len(scenarios_to_plot)
fig, axes = plt.subplots(1, n_scenarios, figsize=(6*n_scenarios, 5))

if n_scenarios == 1:
    axes = [axes]

for idx, scenario in enumerate(scenarios_to_plot):
    ax = axes[idx]
    scenario_df = df_results[df_results['scenario'] == scenario]
    
    # Find min and max for THIS scenario only (for consistent y-axis within scenario)
    scenario_min = scenario_df['time_to_goal'].min()
    scenario_max = scenario_df['time_to_goal'].max()
    y_margin = (scenario_max - scenario_min) * 0.1
    scenario_ylim = [max(0, scenario_min - y_margin), scenario_max + y_margin]
    
    # Create grouped bar plot
    x = np.arange(len(gamma_values) * len(epsilon_values))
    width = 0.25
    
    # Define planner order: Particle, Worst, CC-MPC
    planner_order = ['Particle', 'Worst', 'CC-MPC']
    planners = [p for p in planner_order if p in scenario_df['planner'].values]
    
    for p_idx, planner in enumerate(planners):
        planner_df = scenario_df[scenario_df['planner'] == planner].sort_values(['gamma', 'epsilon'])
        positions = x + (p_idx - 1) * width
        
        ax.bar(positions, planner_df['time_to_goal'], width, label=planner, alpha=0.8)
    
    # Set labels and title
    ax.set_xlabel('Gamma & Epsilon', fontsize=10)
    ax.set_ylabel('time_to_goal', fontsize=10)
    ax.set_title(f'{scenario}\n[y: {scenario_ylim[0]:.2f} - {scenario_ylim[1]:.2f}]', 
                 fontsize=12, fontweight='bold')
    ax.legend()
    
    # Set y-axis range specific to this scenario
    ax.set_ylim(scenario_ylim)
    
    # Create x-tick labels
    labels = [f'γ={g}, ε={e}' for g in gamma_values for e in epsilon_values]
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax.grid(axis='y', alpha=0.3)

# Add overall title
fig.suptitle('Time to Goal', fontsize=22, fontweight='bold')

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space at top for title
plt.savefig('time_to_goal_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ time_to_goal comparison plot saved as 'time_to_goal_comparison.png'")
print(f"   (Each scenario has its own y-axis range)")


In [ ]:
# === HEATMAPS: GAMMA vs EPSILON EFFECT ON METRICS ===

# Create heatmaps for each planner and scenario combination
for metric in ['min_dist', 'time_to_goal']:
    print(f"\n{'='*100}")
    print(f"HEATMAPS FOR {metric.upper()}")
    print(f"{'='*100}\n")
    
    # Define scenario order
    scenario_order = ['Right-Straight', 'Right-Straight (unknown)', 'Down-Left', 'Down-Right', 'Down-Straight (unknown)']
    scenarios_to_plot = [s for s in scenario_order if s in df_results['scenario'].values]
    n_scenarios = len(scenarios_to_plot)
    
    # Define planner order: Particle, Worst, CC-MPC
    planner_order = ['Particle', 'Worst', 'CC-MPC']
    n_planners = len(planner_order)
    
    fig, axes = plt.subplots(n_planners, n_scenarios, figsize=(5*n_scenarios, 4*n_planners))
    
    if n_planners == 1 and n_scenarios == 1:
        axes = np.array([[axes]])
    elif n_planners == 1:
        axes = axes.reshape(1, -1)
    elif n_scenarios == 1:
        axes = axes.reshape(-1, 1)
    
    # Calculate min/max for each scenario to ensure consistent color scale within scenario
    scenario_ranges = {}
    for scenario in scenarios_to_plot:
        scenario_data = df_results[df_results['scenario'] == scenario][metric]
        scenario_ranges[scenario] = (scenario_data.min(), scenario_data.max())
    
    for p_idx, planner in enumerate(planner_order):
        for s_idx, scenario in enumerate(scenarios_to_plot):
            ax = axes[p_idx, s_idx]
            
            # Get scenario-specific min/max for consistent color scale within this scenario
            vmin, vmax = scenario_ranges[scenario]
            
            # Filter data
            filtered_df = df_results[
                (df_results['planner'] == planner) & 
                (df_results['scenario'] == scenario)
            ]
            
            # Create pivot table for heatmap
            pivot_table = filtered_df.pivot(index='gamma', columns='epsilon', values=metric)
            
            # Create pivot table for min_dist to check collision
            pivot_min_dist = filtered_df.pivot(index='gamma', columns='epsilon', values='min_dist')
            
            # Create custom annotations:
            # - For min_dist: always show the value
            # - For time_to_goal: show "collide" if min_dist < 0.5, else show value
            if metric == 'min_dist':
                # Always show min_dist values
                annot_array = True
            else:
                # For time_to_goal, replace with "collide" if collision occurred
                annot_array = np.empty_like(pivot_table, dtype=object)
                for i, gamma in enumerate(pivot_table.index):
                    for j, epsilon in enumerate(pivot_table.columns):
                        if pivot_min_dist.loc[gamma, epsilon] < 0.5:
                            annot_array[i, j] = 'collide'
                        else:
                            annot_array[i, j] = f'{pivot_table.loc[gamma, epsilon]:.3f}'
            
            # Create heatmap with scenario-specific color scale
            fmt_str = '.3f' if metric == 'min_dist' else ''
            sns.heatmap(pivot_table, annot=annot_array, fmt=fmt_str, 
                       cmap='RdYlGn_r' if metric == 'min_dist' else 'RdYlGn',
                       vmin=vmin, vmax=vmax,
                       ax=ax, cbar_kws={'label': metric},
                       annot_kws={'fontsize': 15})
            
            ax.set_title(f'{planner} - {scenario}\n[{vmin:.2f} - {vmax:.2f}]', 
                        fontsize=11, fontweight='bold')
            ax.set_xlabel('Epsilon', fontsize=10)
            ax.set_ylabel('Gamma (γ)', fontsize=10)
    
    # Add overall title for the figure
    if metric == 'min_dist':
        fig.suptitle('Minimum Distance', fontsize=24, fontweight='bold')
    else:
        fig.suptitle('Time to Goal', fontsize=24, fontweight='bold')
    
    plt.tight_layout(rect=[0, 0, 1, 0.97])  # Leave space at top for title
    plt.savefig(f'{metric}_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"✅ {metric} heatmap saved as '{metric}_heatmap.png'")
    print(f"   (Each scenario has its own color scale range)")
    print(f"   Note: Cells with min_dist < 0.5 show 'collide' in red")


In [ ]:
# === DETAILED COMPARISON TABLES BY SCENARIO ===

print("=" * 120)
print("DETAILED COMPARISON: PLANNERS BY SCENARIO")
print("=" * 120)

# Define scenario order
scenario_order = ['Right-Straight', 'Right-Straight (unknown)', 'Down-Left', 'Down-Right', 'Down-Straight (unknown)']
scenarios_to_analyze = [s for s in scenario_order if s in df_results['scenario'].values]

for scenario in scenarios_to_analyze:
    print(f"\n{'#'*120}")
    print(f"# {scenario}")
    print(f"{'#'*120}\n")
    
    scenario_df = df_results[df_results['scenario'] == scenario]
    
    # Show best performers for each metric
    print(f"Best min_dist (lower is better):")
    best_min_dist = scenario_df.nsmallest(5, 'min_dist')[['planner', 'gamma', 'epsilon', 'min_dist', 'time_to_goal']]
    print(best_min_dist.to_string(index=False))
    
    print(f"\nBest time_to_goal (lower is better):")
    best_time = scenario_df.nsmallest(5, 'time_to_goal')[['planner', 'gamma', 'epsilon', 'min_dist', 'time_to_goal']]
    print(best_time.to_string(index=False))
    
    # Summary by planner (in order: Particle, Worst, CC-MPC)
    print(f"\nSummary by Planner:")
    print("-" * 120)
    planner_order = ['Particle', 'Worst', 'CC-MPC']
    for planner in [p for p in planner_order if p in scenario_df['planner'].values]:
        planner_df = scenario_df[scenario_df['planner'] == planner]
        print(f"\n{planner}:")
        print(f"  min_dist     - Mean: {planner_df['min_dist'].mean():.4f}, Std: {planner_df['min_dist'].std():.4f}, Min: {planner_df['min_dist'].min():.4f}, Max: {planner_df['min_dist'].max():.4f}")
        print(f"  time_to_goal - Mean: {planner_df['time_to_goal'].mean():.4f}, Std: {planner_df['time_to_goal'].std():.4f}, Min: {planner_df['time_to_goal'].min():.4f}, Max: {planner_df['time_to_goal'].max():.4f}")
    
    print("\n")

# Overall best performers across all scenarios
print("\n" + "=" * 120)
print("OVERALL BEST PERFORMERS (ACROSS ALL SCENARIOS)")
print("=" * 120)

print(f"\nTop 10 by min_dist:")
top_min_dist = df_results.nsmallest(10, 'min_dist')[['scenario', 'planner', 'gamma', 'epsilon', 'min_dist', 'time_to_goal']]
print(top_min_dist.to_string(index=False))

print(f"\nTop 10 by time_to_goal:")
top_time = df_results.nsmallest(10, 'time_to_goal')[['scenario', 'planner', 'gamma', 'epsilon', 'min_dist', 'time_to_goal']]
print(top_time.to_string(index=False))


In [ ]:
# === ACCESS SPECIFIC EXPERIMENT RESULTS ===

# Customize these parameters to inspect a specific experiment
scenario_inspect = "Scenario1"
planner_inspect = "MPPI-Predict"  # Options: "MPPI-Predict", "MPPI-Worst", "Probabilistic"
gamma_inspect = 1
epsilon_inspect = 0.05

key = f"{scenario_inspect}_{planner_inspect}_gamma{gamma_inspect}_eps{epsilon_inspect}"

if key in results_sweep:
    print("=" * 120)
    print(f"DETAILED RESULTS FOR: {key}")
    print("=" * 120)
    
    result = results_sweep[key]
    
    # Show configuration
    print(f"\nConfiguration:")
    print(f"  Scenario: {scenario_inspect}")
    print(f"  Planner: {planner_inspect}")
    print(f"  Gamma: {gamma_inspect}")
    print(f"  Epsilon: {epsilon_inspect}")
    
    # Show results structure
    print(f"\nResult structure:")
    if isinstance(result, dict):
        print(f"  Type: Dictionary with {len(result)} keys")
        print(f"  Keys: {list(result.keys())}")
        
        print(f"\nKey Metrics:")
        for k, v in result.items():
            if not isinstance(v, (list, np.ndarray, dict)):
                print(f"  {k}: {v}")
            elif isinstance(v, (list, np.ndarray)):
                print(f"  {k}: {type(v).__name__} with length {len(v)}")
            else:
                print(f"  {k}: {type(v).__name__}")
    else:
        print(f"  Type: {type(result)}")
        print(f"  Value: {result}")
else:
    print(f"❌ Key '{key}' not found in results_sweep")
    print("\nAvailable keys for this scenario:")
    for k in sorted(results_sweep.keys()):
        if k.startswith(scenario_inspect):
            print(f"  - {k}")

# Show available combinations
print("\n" + "=" * 120)
print("ALL AVAILABLE EXPERIMENT KEYS (grouped by scenario and planner)")
print("=" * 120)

for scenario in sorted(scenarios, key=lambda x: x['name']):
    scenario_name = scenario['name']
    print(f"\n{scenario_name}:")
    
    for planner_cfg in planner_configs:
        planner_name = planner_cfg['name']
        print(f"  {planner_name}:")
        
        count = 0
        for key in sorted(results_sweep.keys()):
            if key.startswith(f"{scenario_name}_{planner_name}"):
                count += 1
        print(f"    {count} experiments")
